In [1]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
# Load Dataset
df = pd.read_csv('netflix_titles.csv')
# DATASET FROM https://www.kaggle.com/datasets/shivamb/netflix-shows?resource=download

# Show top 3 rows
df.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [55]:
# Get shape of dataset
df.shape

(8807, 12)

In [56]:
# List important columns
columns = ['cast','director','listed_in','title','country']

In [57]:
# Show data with important columns
df[columns].head(2)

,cast,director,listed_in,title,country
0,NaN,Kirsten Johnson,Documentaries,Dick Johnson Is Dead,United States
1,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",NaN,"International TV Shows, TV Dramas, TV Mysteries",Blood & Water,South Africa


In [58]:
# Show new data shape
df[columns].shape

(8807, 5)

In [59]:
# Check for missing values
df[columns].isnull().values.sum()

4290

In [60]:
# Show new data shape
df.dropna(axis='rows')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...
...,...,...,...,...,...,...,...,...,...,...,...,...
8801,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [61]:
df[columns].shape

(8807, 5)

In [62]:
# Check for missing values
df[columns].isnull().values.any()

True

In [66]:
# Function to combine all features into one string

def getFeatures(data):
    features = []
    for i in range(0, data.shape[0]):
#         features.append(data['title'][i])
        features.append(str(data['cast'][i]) + ' ' + str(data['director'][i])+ ' ' + str(data['listed_in'][i]) + ' ' + str(data['title'][i]) + ' '+ str(data['country'][i]))
#         features.append(data['cast'][i] + ' ' + data['director'][i] + ' ' + data['listed_in'][i] + ' ' + data['title'][i])

    return features

In [67]:
# Add all features into one column
df['combinedFeatures'] = getFeatures(df)

df.head(50)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,combinedFeatures
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",nan Kirsten Johnson Documentaries Dick Johnson...
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...","Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi..."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...","nan nan Docuseries, Reality TV Jailbirds New O..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K..."
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,"Kate Siegel, Zach Gilford, Hamish Linklater, H..."
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,"Vanessa Hudgens, Kimiko Glenn, James Marsden, ..."
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...","Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho..."
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T..."


In [68]:
# Text to Vector
cm = CountVectorizer().fit_transform(df['combinedFeatures'])

In [69]:
# cosine similarity 
cs = cosine_similarity(cm)

# print
print(cs)

[[1.         0.0375823  0.04564355 ... 0.12309149 0.11111111 0.        ]
 [0.0375823  1.         0.26760077 ... 0.         0.         0.05106429]
 [0.04564355 0.26760077 1.         ... 0.         0.         0.03100868]
 ...
 [0.12309149 0.         0.         ... 1.         0.20515248 0.0418121 ]
 [0.11111111 0.         0.         ... 0.20515248 1.         0.03774257]
 [0.         0.05106429 0.03100868 ... 0.0418121  0.03774257 1.        ]]


In [70]:
# Cosine similarty shape
cs.shape

(8807, 8807)

In [123]:
# Similarity for titile
title = "The Lord of the Rings: The Two Towers"

# get movie's id
show_id = df[df.title == title]['show_id'].values[0]
show_id = show_id[1::]
print(show_id)

8406


In [124]:
# Create list of similarity scores
scores = list(enumerate(cs[int(show_id)+1]))

print(scores[1])

(1, 0.06315089779506727)


In [125]:
# sort list
sortedList = sorted(scores, key = lambda x:x[1], reverse = True)
sortedList = sortedList[1:]

In [126]:
# Print sorted list
print(sortedList)

[(283, 0.6054055145966812), (8619, 0.5716619504750295), (8294, 0.560112033611204), (8521, 0.560112033611204), (91, 0.5457051563317492), (8561, 0.53813823519705), (8268, 0.5294117647058824), (8516, 0.5185629788417315), (7000, 0.5144957554275266), (8707, 0.5118906968889915), (2097, 0.5009794328681195), (7016, 0.5009794328681195), (6273, 0.5007733956671915), (3479, 0.4950737714883372), (2805, 0.4900980294098035), (3118, 0.4900980294098035), (4258, 0.49009802940980346), (4606, 0.49009802940980346), (3349, 0.48507125007266594), (6938, 0.48507125007266594), (7076, 0.4850712500726658), (8462, 0.48082236894595093), (8350, 0.4756514941544941), (2384, 0.47162110914189925), (8538, 0.4708709557974187), (645, 0.47087095579741867), (724, 0.47087095579741867), (1744, 0.47087095579741867), (7699, 0.47087095579741867), (8744, 0.47087095579741867), (2316, 0.47058823529411764), (8450, 0.47058823529411764), (3987, 0.4653789209955172), (8479, 0.46537892099551714), (3529, 0.4601789933084223), (5925, 0.46017

In [127]:
# Print out top 5 similar shows/movies
j = 0
print('Top 5 most recommended movies/shows to', title, 'are:\n')
for item in sortedList:
    name = ('s'+str(item[0]))
    movieTitle = df[df.show_id==name]['title'].values[0]
    print(j+1, movieTitle)
    j = j+1
    if j>4:
        break

Top 5 most recommended movies/shows to The Lord of the Rings: The Two Towers are:

1 La diosa del asfalto
2 Trash Fire
3 The Fear
4 The Student
5 Paradise Hills
